<a href="https://colab.research.google.com/github/Krish77na/Amazon-review-s-Summarization/blob/main/Exp_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer
import pandas as pd

# Load the dataset
data = pd.read_csv('Reviews[1].csv', nrows=100000, error_bad_lines=False)

# Initialize the T5 model and tokenizer
model = T5ForConditionalGeneration.from_pretrained('t5-base')
tokenizer = T5Tokenizer.from_pretrained('t5-base')

def summarize_text(text):
    input_text = "summarize: " + text
    input_ids = tokenizer.encode(input_text, return_tensors='pt', max_length=512, truncation=True)
    summary_ids = model.generate(input_ids, max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

# Process and print summaries for the first few rows
num_rows_to_process = 20  # You can adjust this number
for i in range(num_rows_to_process):
    original_text = data['Text'][i]
    original_summary = data['Summary'][i]
    generated_summary = summarize_text(original_text)
    print(f"Original Text: {original_text}\nOriginal Summary: {original_summary}\nGenerated Summary: {generated_summary}\n")


<ipython-input-4-412b840ee1f2>:6: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  data = pd.read_csv('Reviews[1].csv', nrows=100000, error_bad_lines=False)
/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5.py:240: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(

Original Text: I have bought several of the Vitality canned dog food products and have found them all to be of good quality. The product looks more like a stew than a processed meat and it smells better. My Labrador is finicky and she appreciates this product better than  most.
Original Summary: Good Quality Dog Food
Generated Summary: my Labrador is finicky and she appreciates this product better than most. the product looks more like a stew than a processed meat and it smells better.

Original Text: Product arrived labeled as Jumbo Salted Peanuts...the peanuts were actually small sized unsalted. Not sure if this was an error or if the vendor intended to represent the product as "Jumbo".
Original Summary: Not as Advertised
Generated Summary: the product arrived labeled as Jumbo Salted Peanuts...the peanuts were actually small sized unsalted. not sure if this was an error or if the vendor intended to represent the product as "Jumbo"

Original Text: This is a confection that has been ar

In [5]:
!pip install nltk rouge


In [14]:
import nltk
from rouge import Rouge
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.meteor_score import meteor_score
from nltk.tokenize import word_tokenize

# Ensure that NLTK's required resources are downloaded
nltk.download('punkt')
nltk.download('wordnet')

def calculate_scores(original_summaries, generated_summaries):
    bleu_scores, meteor_scores, rouge_scores = [], [], []
    rouge = Rouge()

    for original, generated in zip(original_summaries, generated_summaries):
        # Tokenize the sentences for BLEU and METEOR
        original_tokens = word_tokenize(original)
        generated_tokens = word_tokenize(generated)

        # BLEU Score
        bleu_scores.append(sentence_bleu([original_tokens], generated_tokens, weights=(0.25, 0.25, 0.25, 0.25)))

        # METEOR Score
        meteor_scores.append(meteor_score([original_tokens], generated_tokens))

        # ROUGE Score
        original_str = ' '.join(original_tokens)
        generated_str = ' '.join(generated_tokens)
        scores = rouge.get_scores(generated_str, original_str)
        rouge_scores.append(scores[0])

    avg_bleu = sum(bleu_scores) / len(bleu_scores)
    avg_meteor = sum(meteor_scores) / len(meteor_scores)
    avg_rouge = {
        'rouge-1': sum(score['rouge-1']['f'] for score in rouge_scores) / len(rouge_scores),
        'rouge-2': sum(score['rouge-2']['f'] for score in rouge_scores) / len(rouge_scores),
        'rouge-l': sum(score['rouge-l']['f'] for score in rouge_scores) / len(rouge_scores)
    }

    return avg_bleu, avg_meteor, avg_rouge

# Assuming you have lists of original summaries and generated summaries
original_summaries = data['Summary'].tolist()[:20]
generated_summaries = [summarize_text(text) for text in data['Text'].tolist()[:20]]

# Calculate scores
bleu, meteor, rouge = calculate_scores(original_summaries, generated_summaries)

print(f"Average BLEU Score: {bleu}")
print(f"Average METEOR Score: {meteor}")
print(f"Average ROUGE Scores: {rouge}")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Average BLEU Score: 6.599021204148359e-156
Average METEOR Score: 0.14145968133253833
Average ROUGE Scores: {'rouge-1': 0.06129745938166494, 'rouge-2': 0.011251293417730132, 'rouge-l': 0.06129745938166494}


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_